In [1]:
# import libraries
import os

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.metrics import mean_squared_error, classification_report
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("res")

In [3]:
# load data
df = pd.read_csv("Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021_20231012.csv")
# https://www.kaggle.com/datasets/bhautikmangukiya12/hospital-inpatient-discharges-dataset 